In [1]:
#hyper-parameters optimisation for birdirectional(Long Short Term Memory)#!pip install keras-self-attention
from keras_self_attention import SeqSelfAttention
import keras
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import LSTM, Bidirectional
import  os

from keras.models import Sequential, load_model

from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from keras.regularizers import L1L2
from sklearn.preprocessing import MinMaxScaler
import  pandas as pd
import numpy as np

forword=15
look_back=60
tr_size=0.86


df=pd.read_excel('test_data.xlsx')
df1=df.copy()


df1=df1[['日期（月度）','USA_output','OPEC_output','demand_current','supply_current']]
#df1=df1.bfill()
df1=df1.dropna()


df1=df1.rename(columns={"日期（月度）":'日期（日期）'})
df1=df1.set_index('日期（日期）')


df1=df1.asfreq(freq='d')

df1=df1.bfill()
#print(df1.head(40))

df=df.set_index('日期（日期）')


df=df[['Brent','USD','CRB','Premium','trade','BDI']]


df=df.ffill()
df=df.bfill()



df=df.join(df1)

df=df.dropna()
df=df.astype('float64')

df_raw=df.copy()
df = df.diff().dropna()

print(df.shape[0])
print(df_raw.shape[0])




#print(df)



#df=df.iloc[0:50]



dataset=df.values
#dataset = dataset.astype('float32')
#print(dataset)
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)






train_size = int(len(dataset) * tr_size)
trainlist = dataset[:train_size]
testlist = dataset[train_size:]



def create_dataset(dataset, look_back,forword):

        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-forword):
            a = dataset[i:(i+look_back)]
            dataX.append(a)
            dataY.append(dataset[(i + look_back):(i+look_back+forword)])
        return np.array(dataX),np.array(dataY)


trainX,trainY  = create_dataset(trainlist,look_back,forword)
testX,testY = create_dataset(testlist,look_back,forword)


trainY=np.reshape(trainY,(trainY.shape[0],-1))


def retrive(hat):
    hat=np.reshape(hat,(-1,df.shape[1]))
    hat=scaler.inverse_transform(hat)
    hat=np.reshape(hat,(-1,forword,df.shape[1]))
    ind=train_size+look_back-1
    result=[]
    for y in range(hat.shape[0]):
        ind_temp=ind+y
        temp=np.array([ hat[y][x][0] for x in range(forword)])
        las=df_raw.iloc[(ind_temp+1)]['Brent']
        temp=np.insert(temp,0,las)
        temp=np.cumsum(temp)
        result.append(temp[1:])
    #print(np.array(result))
    return np.array(result)

y_glob=retrive(testY)

#print(trainX)
#print(trainY)


space= { 
                      'units3': hp.quniform('units3',64,512,1),
                      'dropout3': hp.uniform('dropout3',0.25,0.75),
    'rec3': hp.uniform('rec',0.25,0.75),
 
                                        
'batch_size' : hp.quniform('batch_size',28,128,1),
'units1':hp.quniform('units1',64,512,1),

'dropout1':hp.uniform('dropout1',0.25,0.75),
    'rec1': hp.uniform('rec1',0.25,0.75),
    'ker1_1': hp.uniform('ker1_1',0.01,0),
    'ker1_2': hp.uniform('ker1_2',0.01,0),
    'ker1_3': hp.uniform('ker1_3',0.01,0),
    'ker1_4': hp.uniform('ker1_4',0.01,0),
}


def trainModel(params):

    model = Sequential()
    model.add(Bidirectional(LSTM(int(params['units1']), dropout=params['dropout1'], recurrent_dropout=params['rec1'],kernel_regularizer=L1L2(l1=params['ker1_1'], l2=params['ker1_2']), recurrent_regularizer=L1L2(l1=0.001, l2=0.001), bias_regularizer=L1L2(l1=0.001, l2=0.001),return_sequences=True), input_shape=(trainX.shape[1], trainX.shape[2])))
    model.add(Bidirectional(LSTM(int(params['units3']), dropout=params['dropout3'], recurrent_dropout=params['rec3'],kernel_regularizer=L1L2(l1=params['ker1_3'], l2=params['ker1_4']), recurrent_regularizer=L1L2(l1=0.001, l2=0.001), bias_regularizer=L1L2(l1=0.001, l2=0.001),return_sequences=True)))
    model.add(SeqSelfAttention(attention_activation='sigmoid',kernel_regularizer=L1L2(l1=0.0001, l2=0.0001),bias_regularizer=L1L2(l1=0.0001, l2=0.0001),attention_regularizer_weight=1e-4))

    model.add(Flatten())
    model.add(Dense(trainY.shape[1]))
    model.add(Activation("relu"))
    model.compile(loss='mse', optimizer='adam')
    
    model.fit(trainX, trainY, epochs=30, batch_size=int(params['batch_size']), verbose=2)
    y_hat  =  model.predict(testX)



    diff=retrive(y_hat)-y_glob

    #rmse=np.sqrt((diff**2).mean())
    mae=(np.abs(diff)).mean()
    return {'loss': mae, 'status': STATUS_OK}



trials = Trials()
best = fmin(trainModel, space, algo=tpe.suggest, max_evals=45, trials=trials)
fo = open("res.txt", "w")
fo.write( repr(best))
fo.close()

  Created wheel for keras-self-attention: filename=keras_self_attention-0.41.0-cp36-none-any.whl size=17289 sha256=968fc144e3d8dd53a31b18438af315edea6dabede07c700ff80aa47f995c6fff
  Stored in directory: /tmp/.cache/pip/wheels/cc/dc/17/84258b27a04cd38ac91998abe148203720ca696186635db694
Successfully built keras-self-attention


Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

6574
6575
Epoch 1/30
 - 53s - loss: 41.8254

Epoch 2/30
 - 44s - loss: 3.0110

Epoch 3/30
 - 44s - loss: 2.6332

Epoch 4/30
 - 44s - loss: 2.3101

Epoch 5/30
 - 44s - loss: 2.0306

Epoch 6/30
 - 44s - loss: 1.7900

Epoch 7/30
 - 43s - loss: 1.5939

Epoch 8/30
 - 44s - loss: 1.4308

Epoch 9/30
 - 43s - loss: 1.2920

Epoch 10/30
 - 44s - loss: 1.1797

Epoch 11/30
 - 44s - loss: 1.1133

Epoch 12/30
 - 44s - loss: 1.1172

Epoch 13/30
 - 43s - loss: 1.1259

Epoch 14/30
 - 43s - loss: 1.1301

Epoch 15/30
 - 43s - loss: 1.1313

Epoch 16/30
 - 43s - loss: 1.1322

Epoch 17/30
 - 44s - loss: 1.1336

Epoch 18/30
 - 43s - loss: 1.1356

Epoch 19/30
 - 44s - loss: 1.1373

Epoch 20/30
 - 43s - loss: 1.1377

Epoch 21/30
 - 43s - loss: 1.1374

Epoch 22/30
 - 43s - loss: 1.1370

Epoch 23/30
 - 43s - loss: 1.1367

Epoch 24/30
 - 43s - loss: 1.1366

Epoch 25/30
 - 43s - loss: 1.1369

Epoch 26/30
 - 44s - loss: 1.1368

Epoch 27/30
 - 43s - loss: 1.1368

Epoch 28/30
 - 43s - loss: 1.1370

Epoch 29/30
 - 43s

AssertionError: 